# Preprocessing data

### Imports

In [ ]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
import seaborn as sns
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import keras.utils
import tensorflow as tf
import requests
from time import sleep
import pandas as pd
import re
from matplotlib import pyplot as plt
from random import randint
import re
!pip3 install instaloader
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile
import csv
!pip install gradio
import gradio as gr
import pickle
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

%matplotlib notebook

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Loading dataset

In [ ]:
!unzip '/content/labeled_data.csv (3).zip' -d '/content/sample_data'

Archive:  /content/labeled_data.csv (3).zip
  inflating: /content/sample_data/labeled_data.csv  


In [ ]:
HSdataset = pd.read_csv('/content/sample_data/labeled_data.csv')
HSdataset.describe()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000


In [ ]:
HSdataset[HSdataset['class']==0].sample(20)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
4451,4582,3,2,1,0,0,@QUAN1T0 You retarded bruh lol
23038,23521,3,2,1,0,0,Y'all hoes really get mad cause a nigga don't ...
17943,18345,3,2,1,0,0,RT @Unorthodex: Nah I hate white people now si...
5120,5272,3,2,1,0,0,@Walmart fuck your store bitch
7756,7974,3,2,1,0,0,And the Jew @Deuterostomes favorited my tweet ...
18087,18491,3,2,0,1,0,RT @Woody_AFC: I saw a great chimpout 2 summer...
2853,2923,3,3,0,0,0,@Christian_ESEM you don't know what your talki...
9178,9434,3,2,1,0,0,Fredo was a bitch ass nigga. How niggas fire o...
18456,18865,3,2,1,0,0,RT @_iHATEMOON: All these bitches &amp; niggaz...
7877,8098,3,2,1,0,0,Ayy Carlton is a ghetto school now #SwitchedAt...


### Dataset cleaning

In [ ]:
HSdataset.rename(columns={'tweet':'text'}, inplace=True)
HSdataset = HSdataset.drop(['count', 'offensive_language', 'neither', 'hate_speech', 'Unnamed: 0'], axis=1)
HSdataset['class_new'] = HSdataset['class'].map({0: 0, 1: 0,2: 1})
HSdataset.sample(40)

,class,text,class_new
16658,2,RT @NinerFans: #Seahawks fans are scheduled to...,1
9632,1,Hahahaha fucking pussies @kevinlove @KingJames...,0
14012,2,RT @23Vnds: Check out the homie @renz360 for t...,1
20913,1,Smokin on loud that will have yo ass panicking...,0
19219,1,"RT @iam_KandiRaven: ""You gotta be able to keep...",0
24151,1,lil bitch,0
2632,1,"@BlakkHoodie_562 yeah that niggah played good,...",0
7633,1,All these bitches wanna be like Kim K and fuck...,0
23533,1,actin awfully slutty for a bitch in a winnie t...,0
4246,1,@NataIiaxxx @ladygaga suck on my throbbing pussy,0


### Normalization

In [ ]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
re1 = re.compile(r"^@.*\s$")
re2 = re.compile(r"\S*https?:\S*")
re3 = re.compile(r"[^a-zA-Z\s]+")
re4 = re.compile(r"rt")
re5 = re.compile(r"\n")
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def clean_text(text):
#zmiana liter na małe
  text = str(text).lower()
#usunięcie nicku -> @...
  text = re1.sub('', text)
#usunięcie linku
  text = re2.sub('', text)
#usunięcie znaków specjalnych i cyfr
  text = re3.sub('', text)
#usunięcie 'rt'
  text = re4.sub('', text)
#usunięcie /n
  text = re5.sub('', text)
#usunięcie stopwords w tokenizowanym tekście
  text = [word for word in text.split(' ') if word not in stopwords.words('english')]
  text=" ".join(text)
#Steamming
  text = [stemmer.stem(word) for word in text.split(' ')]
  text=" ".join(text)
#lematyzacja
  text = [lemmatizer.lemmatize(word) for word in text.split(' ')]
  text=" ".join(text)
  return text

In [ ]:
HSdataset['text'] = HSdataset['text'].apply(clean_text)

,class,text,class_new
18102,1,xanaxdream u know french inhal ur bad bitch,0
17970,2,vandtrailroad didyouknow comstock lode boon ...,1
7613,1,bitch shameless straight bugoutsss,0
8056,1,bitch may million dollar bodi got dollar menu ...,0
15709,1,jlewyvil treslyon keyair angel amp femal pay...,0
16811,1,onlybiebsd plea bitch im belieb lt,0
2284,1,problem bitch aint,0
4486,1,rana cant hoe,0
3255,1,giantraiderk oh sneaki son bitch,0
10191,1,dont buy pussi cuz free bitch buy lac,0


In [ ]:
HSdataset.sample(20) #można znaleźć wyrazy najbliższe np bihday

,class,text,class_new
19444,1,kingdaddi bitch got red cootche rover shit e...,0
3487,1,imsolk v gun game pussi,0
8796,1,dont fuck girl wear beani,0
7488,1,offici old as bitch,0
13808,2,peopl pictur far wide reason make teabagg poo...,1
5136,1,whatabarb money car cloth hoe lol want success...,0
21818,1,hoe worri bout wrong thing life,0
18268,1,connerjay ho ho ho bitch merri monthsish chr...,0
16551,1,mssadaclay thanx im diet vianawf mssadaclay...,0
13204,1,naughti matur av model sweet pussi spread huge...,0


### Visualization

In [ ]:
sns.countplot(HSdataset['class_new'])

/usr/local/lib/python3.7/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [ ]:
# class count
class_count_0, class_count_1 = HSdataset['class_new'].value_counts()

# Separate class
class_0 = HSdataset[HSdataset['class_new'] == 0]
class_1 = HSdataset[HSdataset['class_new'] == 1] 
# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (20620, 3)
class 1: (4163, 3)


In [ ]:
#Undersampling
class_0_under = class_0.sample(class_count_1)

HSdataset = pd.concat([class_0_under, class_1], axis=0)

print("total class of 1 and 0:",HSdataset['class_new'].value_counts())
# plot the count after under-sampeling
HSdataset['class_new'].value_counts().plot(kind='bar', title='count (target)')

print((HSdataset.duplicated()==True).value_counts())

total class of 1 and 0: 0    4163
1    4163
Name: class_new, dtype: int64
False    8281
True       45
dtype: int64


In [ ]:
from wordcloud import WordCloud
import random
HateData = HSdataset[HSdataset['class_new']==1]

visual_rev = WordCloud().generate(' '.join(HateData['text']))
plt.figure(figsize=(8,8))
plt.imshow(visual_rev, interpolation='bilinear')
plt.show()

<IPython.core.display.Javascript object>

# Basic modelling

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = HSdataset['text']
y = HSdataset['class_new']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

### TF-IDF/Logistic regression/Random Forest

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit(X_train)

TfidfVectorizer()

In [ ]:
#Bag of words
tfidf.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['aaliyahhlove',
 'aan',
 'aap',
 'aaron',
 'aaronwoh',
 'aaryneliza',
 'aarynelizagri',
 'ab',
 'abbey',
 'abbi',
 'abc',
 'abcbird',
 'abcrispyyandi',
 'abdelka',
 'abdullahomar',
 'abdurahman',
 'abel',
 'abelsmasochist',
 'aberdeen',
 'abil',
 'abl',
 'abo',
 'aboion',
 'abound',
 'abouta',
 'abraham',
 'abrehamdadi',
 'abreumegan',
 'abrown',
 'absolut',
 'absoluteyvil',
 'abstractlif',
 'absurd',
 'abt',
 'abu',
 'abus',
 'abuyazidmuawiya',
 'abylecardin',
 'ac',
 'aca',
 'acab',
 'acaciabrinley',
 'acaciamakayla',
 'academ',
 'accent',
 'accept',
 'access',
 'accid',
 'accident',
 'accipit',
 'accipitrida',
 'accompani',
 'accord',
 'account',
 'accus',
 'ace',
 'acehudsonjr',
 'aceptar',
 'acev',
 'achosenword',
 'acid',
 'acino',
 'acknowledg',
 'acmobbdeep',
 'acn',
 'acoust',
 'acquir',
 'across',
 'act',
 'actabunnifoofoo',
 'actin',
 'action',
 'activ',
 'activist',
 'actor',
 'actress',
 'actual',
 'actuallyamdolan',
 'ad',
 'adam',
 'adambaldwin',
 'adamjferraro',
 'adam

In [ ]:
tfidf.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [ ]:
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)
X_train_tf.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_tf, y_train)
y_pred = lr.predict(X_test_tf)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.91      0.94      1249
           1       0.92      0.97      0.94      1249

    accuracy                           0.94      2498
   macro avg       0.94      0.94      0.94      2498
weighted avg       0.94      0.94      0.94      2498



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(X_train_tf, y_train)
y_pred = rfc.predict(X_test_tf)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94      1249
           1       0.93      0.94      0.94      1249

    accuracy                           0.94      2498
   macro avg       0.94      0.94      0.94      2498
weighted avg       0.94      0.94      0.94      2498



In [ ]:
important_features = pd.Series(data=rfc.feature_importances_,index=tfidf.get_feature_names())
important_features.sort_values(ascending=False,inplace=True)
important_features.head(20)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


bitch     0.202980
hoe       0.068416
pussi     0.031519
trash     0.031466
fuck      0.023348
nigga     0.018122
bird      0.017788
yanke     0.015260
as        0.013649
shit      0.013368
yellow    0.011747
charli    0.011475
faggot    0.009065
nigger    0.007128
aint      0.006816
fag       0.005530
dont      0.005147
color     0.004781
oreo      0.004695
jihadi    0.004675
dtype: float64

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf = clf.fit(X_train_tf, y_train)

In [ ]:
clf.score(X_train_tf, y_train)

0.9725463280713795

In [ ]:
clf.score(X_test_tf, y_test)

0.9027221777421938

# Neural network

## Tokenization

In [ ]:
max_words = 10000 #Bierzemy pod uwagę 10000 najczęściej występujących słów w zbiorze
maxlen = 100 #skraca komentarz do 100 słów

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index
print('Znaleziono %s unikatowych tokenów.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(y)
print('Kształt tensora danych:', data.shape)
print('Kształt tensora etykiet:', labels.shape)
print(tokenizer)

Znaleziono 15741 unikatowych tokenów.
Kształt tensora danych: (8326, 100)
Kształt tensora etykiet: (8326,)


In [ ]:
#Train, test, val split of tokenized text
X_train_tkn, X_test_tkn, y_train_tkn, y_test_tkn = train_test_split(data, labels, test_size=0.3, random_state=0, stratify=y)
#X_test_tkn, X_val_tkn, y_test_tkn, y_val_tkn = train_test_split(X_test_tkn, y_test_tkn, test_size=0.3, random_state=0, stratify=y_test_tkn)

## Glove Embedding

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2022-08-30 15:02:23 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Znaleziono %s wektorów słów.' % len(embeddings_index))

Znaleziono 400000 wektorów słów.


In [ ]:
embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Słowa nieznalezione w osadzanym indeksie zostaną zastąpione zerami.
            embedding_matrix[i] = embedding_vector

## Neural network with Glove embedding

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, SpatialDropout1D, LSTM
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='adam' ,metrics=['accuracy']) #adam optimizer

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [ ]:
model.fit(X_train_tkn, y_train_tkn,
                    epochs=5,
                    batch_size=32,
                    validation_data=(X_test_tkn, y_test_tkn))

Epoch 1/5
183/183 [==============================] - 55s 279ms/step - loss: 0.3694 - accuracy: 0.8420 - val_loss: 0.2359 - val_accuracy: 0.9039
Epoch 2/5
183/183 [==============================] - 43s 237ms/step - loss: 0.1936 - accuracy: 0.9327 - val_loss: 0.1938 - val_accuracy: 0.9355
Epoch 3/5
183/183 [==============================] - 43s 236ms/step - loss: 0.1396 - accuracy: 0.9525 - val_loss: 0.1833 - val_accuracy: 0.9408
Epoch 4/5
183/183 [==============================] - 49s 269ms/step - loss: 0.1058 - accuracy: 0.9628 - val_loss: 0.1669 - val_accuracy: 0.9460
Epoch 5/5
183/183 [==============================] - 43s 234ms/step - loss: 0.0776 - accuracy: 0.9737 - val_loss: 0.1750 - val_accuracy: 0.9355


In [ ]:
X_train_tkn

In [ ]:
acc = history1.history['accuracy']
val_acc = history1.history['val_accuracy']
loss = history1.history['loss']
val_loss = history1.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Dokladnosc trenowania', color='m')
plt.plot(epochs, val_acc, 'b', label='Dokladnosc walidacji', color='c')
plt.title('Dokladnosc trenowania i walidacji')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Strata trenowania', color='m')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji', color='c')
plt.title('Strata trenowania i walidacji')
plt.legend()

plt.show()

## Neural network with self embedding

In [ ]:
model2 = Sequential()
model2.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model2.add(SpatialDropout1D(0.2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()
model2.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

history2 = model2.fit(X_train_tkn, y_train_tkn,
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_test_tkn, y_test_tkn))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
183/183 [==============================] - 46s 231ms/step - loss: 0.3199 - accuracy: 0.8854 - val_l

In [ ]:
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
loss = history2.history['loss']
val_loss = history2.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Dokladnosc trenowania', color='m')
plt.plot(epochs, val_acc, 'b', label='Dokladnosc walidacji', color='c')
plt.title('Dokladnosc trenowania i walidacji')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Strata trenowania', color='m')
plt.plot(epochs, val_loss, 'b', label='Strata walidacji', color='c')
plt.title('Strata trenowania i walidacji')
plt.legend()

plt.show()

<IPython.core.display.Javascript object>

# Instagram data

# App


In [ ]:
def make_prediction(comment_text):
    tokenizer.fit_on_texts(comment_text)
    sequences = tokenizer.texts_to_sequences(comment_text)
    data = pad_sequences(sequences, maxlen=maxlen)
    with open("model.pkl", "rb") as f:
        model  = pickle.load(f)
        preds = model.predict([[data]])
    if preds > 0.5:
        return "This comment is hate speech/offensive"
    return "This is neutral comment"
    #print(preds)


#Create the input component for Gradio
comment_input = gr.Textbox(label = "Enter comment")

app = gr.Interface(fn = make_prediction, inputs=comment_input, outputs=gr.Label(num_top_classes=1))
app.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://17148.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces: https://huggingface.co/spaces


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/gradio/routes.py", line 249, in run_predict
    fn_index, raw_input, username, session_state
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 643, in process_api
    predictions, duration = await self.call_function(fn_index, processed_input)
  File "/usr/local/lib/python3.7/dist-packages/gradio/blocks.py", line 556, in call_function
    prediction = await block_fn.fn(*processed_input)
  File "/usr/local/lib/python3.7/dist-packages/gradio/interface.py", line 655, in submit_func
    prediction = await self.run_prediction(args)
  File "/usr/local/lib/python3.7/dist-packages/gradio/interface.py", line 685, in run_prediction
    self.fn, *processed_input, limiter=self.limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/to_thread.py", line 32, in run_sync
    func, *args, cancellable=cancellable, limiter=limiter
  File "/usr/local/lib/python3.7/dist-packages/anyio/_backends/_async

[[0.9571079 ]
 [0.9716745 ]
 [0.97295195]
 [0.95041585]
 [0.9193655 ]
 [0.9819298 ]
 [0.95041585]
 [0.9728755 ]
 [0.9193655 ]
 [0.9257067 ]
 [0.9008098 ]
 [0.9716745 ]
 [0.9575803 ]
 [0.9819298 ]
 [0.9193655 ]
 [0.9193655 ]
 [0.87302035]
 [0.9819298 ]
 [0.9496887 ]
 [0.7761816 ]
 [0.9193655 ]
 [0.9571079 ]
 [0.9716745 ]
 [0.97295195]
 [0.95041585]
 [0.9193655 ]
 [0.89636356]
 [0.96570265]
 [0.9559554 ]
 [0.9728755 ]
 [0.9716745 ]
 [0.9559554 ]
 [0.9193655 ]
 [0.9193655 ]
 [0.9716745 ]
 [0.9008098 ]
 [0.9559554 ]
 [0.97295195]
 [0.9716745 ]
 [0.797559  ]
 [0.9257067 ]
 [0.9559554 ]
 [0.9819298 ]
 [0.9193655 ]
 [0.9193655 ]
 [0.87302035]
 [0.98796487]
 [0.9257067 ]
 [0.97295195]
 [0.93997365]
 [0.9193655 ]
 [0.9728755 ]
 [0.89636356]
 [0.797559  ]
 [0.96570265]
 [0.87302035]
 [0.9193655 ]
 [0.9193655 ]
 [0.9716745 ]
 [0.95041585]
 [0.95041585]
 [0.96570265]
 [0.87302035]
 [0.9559554 ]
 [0.97295195]
 [0.9257067 ]
 [0.9559554 ]
 [0.9716745 ]
 [0.89636356]
 [0.9008098 ]
 [0.9193655 ]
 [0.91

In [ ]:
comm = ['What r u wearing look at that hair hahahhahahhahaha']

make_prediction(comm)


'This comment is hate speech/offensive'

In [ ]:
comm = ['Beyond gorgeous and I love this song by']

make_prediction(comm)

'This is neutral comment'